<a href="https://colab.research.google.com/github/prosyslab/sigpl23-tutorial/blob/main/2_Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경설정

* 라이브러리 설치
* 구글 드라이브 마운트
* 텐서보드 연결

In [ ]:
! pip install transformers datasets evaluate accelerate

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

home_dir = "/content/gdrive/MyDrive/Colab-Data"
model_dir = f"{home_dir}/models/codebert-refinement"

In [6]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir $model_dir

# CodeBERT Fine-tuning 학습하기

### 토크나이저, 데이터셋, 사전학습된 CodeBERT 모델 준비

#### `RobertaLMHeadModel` 모델 구조
* Roberta 모델 + Causal Language Model 구조 사용
* Embedding Layer + 12 x Encoder Layer + Pooler Layer
  * Embedding Layer: batch_size * 514 * 50,265 -> batch_size * 514 * 768
  * Encoder Layer: batch_size * 514 * 768 -> batch_size * 514 * 768
* LM Layer: batch_size * 514 * 768 -> batch_size * 514

In [8]:
import torch

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")

In [10]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("microsoft/codebert-base", is_decoder=True)
print(repr(model))

Some weights of RobertaForCausalLM were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForCausalLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): 

In [ ]:
from datasets import load_dataset

ds = load_dataset("code_x_glue_cc_code_refinement", "small")

In [12]:
ds["train"][0]

{'id': 0,
 'buggy': 'public java.lang.String METHOD_1 ( ) { return new TYPE_1 ( STRING_1 ) . format ( VAR_1 [ ( ( VAR_1 . length ) - 1 ) ] . getTime ( ) ) ; } \n',
 'fixed': 'public java.lang.String METHOD_1 ( ) { return new TYPE_1 ( STRING_1 ) . format ( VAR_1 [ ( ( type ) - 1 ) ] . getTime ( ) ) ; } \n'}

### 데이터 전처리

In [13]:
def tokenize(examples):
  tokenized_inputs = tokenizer(examples["buggy"], padding="max_length", truncation=True)
  labels = tokenizer(examples["fixed"], padding="max_length", truncation=True).input_ids
  return dict(labels=labels, **tokenized_inputs)

tokenized_datasets = ds.map(tokenize, batched=True)
tokenized_datasets

Map:   0%|          | 0/46680 [00:00<?, ? examples/s]

Map:   0%|          | 0/5835 [00:00<?, ? examples/s]

Map:   0%|          | 0/5835 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'buggy', 'fixed', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 46680
    })
    validation: Dataset({
        features: ['id', 'buggy', 'fixed', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 5835
    })
    test: Dataset({
        features: ['id', 'buggy', 'fixed', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 5835
    })
})

### 샘플 데이터 준비

In [14]:
from datasets import DatasetDict

sample = dict()
sample_ratio = 0.05
size = round(tokenized_datasets["train"].num_rows * sample_ratio)
sample["train"] = tokenized_datasets["train"].shuffle(seed=1234).select(range(size))

"""train, validation 에대해서도 동일한 방법으로 데이터를 샘플링할 수 있습니다"""

sample_datasets = DatasetDict(sample)
sample_datasets.num_rows

{'train': 2334, 'validation': 292, 'test': 292}

### 모델 학습 설정 정의


#### 평가식 정의
* [bleu](https://huggingface.co/spaces/evaluate-metric/bleu) 사용
  * 범위: [0-1]
  * 정답과 예측 문자열이 비슷한 정도 측정

In [ ]:
import evaluate
import numpy as np

bleu = evaluate.load("bleu")

def compute_metrics(eval_preds):
  preds, labels = eval_preds
  preds_ids = np.argmax(preds, axis=-1)
  decoded_preds = tokenizer.batch_decode(preds_ids, skip_special_tokens=True)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  res = bleu.compute(predictions=decoded_preds, references=decoded_labels)
  return {"bleu": res["bleu"]}

#### Hyper parameters
실습에서 사용하는 하이퍼파라미터 외에도 실제로 사용되는 하이퍼파라미터가 많습니다. [공식 문서](https://huggingface.co/docs/transformers/main_classes/trainer)를 참고하세요.

* `output_dir`: 모델 저장 위치. 체크포인트, 로그 등 저장
* `evaluation_strategy`: 학습 중 `eval_dataset` 을 이용해서 평가하는 단위
* `save_strategy`: 저장 단위
* `num_train_epochs`: 총 데이터 학습 횟수 지정
* `per_device_train_batch_size`: 학습 데이터 배치 크기. 이 크기에 따라 메모리 사용량이 매우 달라집니다.
* `gradient_accumulation_steps`: 역전파 단위
  * `per_device_train_batch_size * gradient_accumulation_steps` 단위로 역전파되며, 이 크기를 총 `TOTAL_BATCH_SIZE` 라고 부르기도 합니다. 분산 학습일 경우에는 사용하는 GPU 갯수까지 곱해서 사용합니다.
  * 하이퍼파라미터에서 `*_steps` 의이름으로 지정되는 값의 경우 1 step 의 크기는 `TOTAL_BATCH_SIZE` 입니다.
* `per_device_eval_batch_size`: 학습 중 평가 시 사용하는 데이터 배치 크기.
  * 평가 시에는 모델을 업데이트하지 않기 때문에 역전파를 위한 중간 텐서를 유지하지 않고, 따라서 메모리 사용량이 학습 과정에 비해 적습니다. 동일한 메모리를 사용할 때 학습 데이터 배치보다 평가 데이터 배치를 크게 잡을 수 있습니다.
* `learning_rate`: gradient step 크기
* `lr_scheduler_type`: 학습 중 learning rate 를 바꾸는 방법
  * `"linear"`: 전체 학습 횟수에 도달할 때까지 선형으로 감소
  * `"constant"`: 전체 학습 중 일정한 learning rate 유지
  * `"cosine"`: consine 함수에 따라 learning rate 가 진동
  * 이 외 문서 참고
* `warmup_ratio`: 학습 초기에 전체 학습 횟수의 `warmup_ratio` 만큼 동안 지정한 `learning_rate` 까지 선형으로 증가하도록 설정
  * 학습 초기에는 learning rate 가 너무 크면 로컬 옵티멈에 빠지기 쉽습니다.
* `logging_steps`: loss 등의 학습 메트링을 로깅하는 단위
  * `gradient_accumulation_steps * logging_steps` 만큼 데이터를 학습한 뒤 로깅합니다.

In [13]:
from transformers import TrainingArguments

args = TrainingArguments(
  output_dir=model_dir,
  evaluation_strategy="epoch",
  save_strategy="epoch",
  num_train_epochs=3.0,
  per_device_train_batch_size=8,
  gradient_accumulation_steps=32,
  per_device_eval_batch_size=16,
  learning_rate=2e-5,
  lr_scheduler_type="linear",
  warmup_ratio=0.1,
  logging_steps=1,
  seed=1234,
)


### 샘플 데이터에서 학습해보기

In [14]:
from transformers import Trainer

trainer = Trainer(
  model=model,
  args=args,
  train_dataset=sample_datasets["train"],      # 학습 데이터
  eval_dataset=sample_datasets["validation"],  # 평가 데이터
)
trainer.train(resume_from_checkpoint=None)
trainer.save_model(args.output_dir)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,10.465800,9.282316
1,6.196800,5.454304
2,4.684200,4.113577


### 테스트 데이터에서 정확도 검토하기

※ 참고: CodeXGLUE 리더보드 https://microsoft.github.io/CodeXGLUE/
* CodeXGLUE 벤치마크 평가 시에는 생성 시간에 Beam search 가 적용되어있어 우리가 정의한 `compute_metrics` 와 차이가 있습니다.
* Beam search 가 적용된 생성 품질을 평가 해보고 싶을 경우 CodeXGLUE 가 제공하는 평가 스크립트 사용해보세요
* 일반적으로 Beam search 를 적용해서 Seq2Seq 모델을 학습하고 평가하고싶을 경우 `Seq2SeqTrainer` 를 사용할 수 있습니다.

In [15]:
from torch.utils.data import DataLoader

predicts = []
labels = []
ds_test = sample_datasets["test"]
for batch in DataLoader(ds_test, batch_size=32):
  input_ids = torch.stack(batch["input_ids"], dim=1).to(device)
  attention_mask = torch.stack(batch["attention_mask"], dim=1).to(device)
  with torch.no_grad():
    model_out = model(input_ids=input_ids, attention_mask=attention_mask)
    batch_preds = tokenizer.batch_decode(torch.argmax(model_out.logits, dim=-1).detach(), skip_special_tokens=True)
  batch_labels = tokenizer.batch_decode(torch.stack(batch["labels"], dim=-1), skip_special_tokens=True)
  predicts.extend(batch_preds)
  labels.extend(batch_labels)
  torch.cuda.empty_cache()

bleu.compute(predictions=predicts, references=labels)

{'bleu': 0.0,
 'precisions': [0.018002322880371662, 8.397715821296607e-05, 0.0, 0.0],
 'brevity_penalty': 1.0,
 'length_ratio': 1.77538846748656,
 'translation_length': 24108,
 'reference_length': 13579}

### 저장된 모델 읽어서 실행해보기

In [15]:
import torch
from transformers import RobertaForCausalLM

ds_test = tokenized_datasets['test']

model = RobertaForCausalLM.from_pretrained(model_dir)
model.eval()
model.to(device)

RobertaForCausalLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): 

In [16]:
ex = ds_test[0]
model_out = model("""모델 입력을 만들어 보세요""")
tokenizer.batch_decode(torch.argmax(model_out.logits, dim=-1).detach(), skip_special_tokens=True)

[' extrapdid ts catch MAX MAX ts ts ratherceiver339 formerCapWh Share catch ts MAXceiver instead former cap max339renticesADST max tag ts catch MAX max MAX catch catchADAD 1 max Share ts max max max well ts former max maxMax ts" catch catch deployments AVG,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,']